# GraphRAG Relevance, Factuality and Synthesis Evaluation

## Step 0: environment set up

In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index
import pandas as pd
from conn import connect2Googlesheet,retrieval_rel_docs, get_concatenate_df, apply_metric
# Force reload of the .env file
load_dotenv()

True

In [2]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

Connected to Neo4j database successfully.


## Step 1: Create vector index

In [3]:
create_vector_index(graph, "entities")

Index 'entities' exists but has incorrect dimensions: None. Recreating...


## Step 2: Load questions from google sheet

In [4]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)  

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(data)
df_MedQ.head()

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blo...,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated wit...,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does...",Maybe this question: In patients with moderate...,fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have ...,Local question (not sure if this is the aim of...,fixed,Wrong concept since PEEP by itself is mandator...,Does the use of neuromuscular blockers in pati...
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservat...",Local question (not sure if this is the aim of...,fixed,Check if studies defined conservative by CVP <...,


## Step 3: Relevance check for top K questions

In [ ]:
topk = 5
results_df = retrieval_rel_docs(graph, df_MedQ, top_k=topk)
results_df.to_csv('./outputs/retrieved_docs_results.csv', index=False)

In [8]:
# load the relevant documents
topk = 5
results_df = pd.read_csv('./outputs/retrieved_docs_results.csv')
relevant_docs_df = pd.read_csv('./outputs/relevant_docs.csv').rename(columns={'Docs': 'Relevant Docs'})
concatenated_df = get_concatenate_df(results_df, relevant_docs_df, topk)
concatenated_df.head()

,Question,Relevant Docs,Generated Docs
0,Relevace Q1,ACURASYS ROSE,ACURASYS
1,Relevace Q2,ACURASYS ROSE,ACURASYS
2,Relevace Q3,ROSE,ACURASYS
3,Relevace Q4,ROSE,ACURASYS ROSE
4,Relevace Q5,FACTT,ACURASYS ARDS


In [9]:
# Display the concatenated DataFrame with metrics
concatenated_df = apply_metric(concatenated_df)
#concatenated_df['Odds'] = (concatenated_df['Precision'] / 1 - concatenated_df['Precision'])* (1 - concatenated_df['Recall'] / concatenated_df['Recall'])
# caculate the odds = correctly retrieved and relevant / incorrectly retrieved and relevant
concatenated_df
# Save the concatenated DataFrame to a CSV file
#concatenated_df.to_csv('./outputs/concatenated_results.csv', index=False)

,Question,Relevant Docs,Generated Docs,Accuracy,Precision,Recall
0,Relevace Q1,ACURASYS ROSE,ACURASYS,0.5,1.0,0.5
1,Relevace Q2,ACURASYS ROSE,ACURASYS,0.5,1.0,0.5
2,Relevace Q3,ROSE,ACURASYS,0.0,0.0,0.0
3,Relevace Q4,ROSE,ACURASYS ROSE,0.5,0.5,1.0
4,Relevace Q5,FACTT,ACURASYS ARDS,0.0,0.0,0.0
